# A1111 Extension Installer (Defaults + Recommended)

Installs **Automatic1111** extensions into the WebUI **code tree** so they load properly:

- Code extensions dir: `/opt/stable-diffusion-webui/extensions`
- Data/models root: `/workspace/a1111` (place ControlNet/AnimateDiff/InsightFace/etc. models here)

**Behavior**
- **Defaults** (your curated list) are **installed automatically** when this cell runs
- **Recommended** can be toggled on/off and installed via a button
- You can also paste **extra Git repos** (one per line) and install them

After installing, open WebUI → **Settings → Reload UI** or click **Restart A1111** button below.

In [ ]:
import ipywidgets as W
from IPython.display import display, clear_output, HTML
from pathlib import Path
from urllib.parse import urlparse
import subprocess, os

# ---- Paths ----
CODE_EXT_DIR = Path("/opt/stable-diffusion-webui/extensions")  # MUST be under code tree
DATA_ROOT    = Path("/workspace/a1111")                         # models live here if needed
CODE_EXT_DIR.mkdir(parents=True, exist_ok=True)
(DATA_ROOT / "models").mkdir(parents=True, exist_ok=True)

",
        "# ---- Your lists ----\n",
        "DEFAULT_EXTS = [\n",
        "    {\"name\": \"Aspect Ratio\",       \"url\": \"https://github.com/alemelis/sd-webui-ar\"},\n",
        "    {\"name\": \"Tag Autocomplete\",   \"url\": \"https://github.com/DominikDoom/a1111-sd-webui-tagcomplete\"},\n",
        "    {\"name\": \"Images Browser\",     \"url\": \"https://github.com/AlUlkesh/stable-diffusion-webui-images-browser\"},\n",
        "    {\"name\": \"Config Presets\",     \"url\": \"https://github.com/Zyin055/Config-Presets\"}\n",
        "]\n",
        "\n",
        "RECOMMENDED = [\n",
        "    {\"name\": \"ControlNet\",            \"url\": \"https://github.com/Mikubill/sd-webui-controlnet\",                 \"enable\": False},\n",
        "    {\"name\": \"ADetailer\",             \"url\": \"https://github.com/Bing-su/adetailer\",                            \"enable\": True},\n",
        "    {\"name\": \"Ultimate SD Upscale\",   \"url\": \"https://github.com/Coyote-A/ultimate-upscale-for-automatic1111\", \"enable\": True},\n",
        "    {\"name\": \"Dynamic Prompts\",       \"url\": \"https://github.com/adieyal/sd-dynamic-prompts\",                   \"enable\": False},\n",
        "    {\"name\": \"Openpose Editor\",       \"url\": \"https://github.com/fkunn1326/openpose-editor\",                    \"enable\": False}\n",
        "]\n",
        "\n",
        "# ---- Helpers ----\n",
        "def run(cmd, cwd=None):\n",
        "    print('$', ' '.join(cmd))\n",
        "    return subprocess.run(cmd, cwd=cwd, check=False)\n",
        "\n",
        "def repo_name_from_url(url: str) -> str:\n",
        "    path = urlparse(url).path.rstrip('/')\n",
        "    name = path.split('/')[-1]\n",
        "    return name[:-4] if name.endswith('.git') else name\n",
        "\n",
        "def install_or_update(url: str, base: Path) -> Path:\n",
        "    name = repo_name_from_url(url)\n",
        "    dst  = base / name\n",
        "    if not dst.exists():\n",
        "        print(f"\n[install] {name} ← {url}")\n",
        "        run(['git', 'clone', '--depth', '1', url, str(dst)])\n",
        "    else:\n",
        "        print(f"\n[update] {name}")\n",
        "        run(['git', '-C', str(dst), 'reset', '--hard'])\n",
        "        run(['git', '-C', str(dst), 'pull', '--ff-only'])\n",
        "    return dst\n",
        "\n",
        "def installed_in_code(url_or_name: str) -> bool:\n",
        "    key = repo_name_from_url(url_or_name) if url_or_name.startswith('http') else url_or_name\n",
        "    for p in CODE_EXT_DIR.iterdir():\n",
        "        if p.is_dir() and key.lower() in p.name.lower():\n",
        "            return True\n",
        "    return False\n",
        "\n",
        "# ---- UI Pieces ----\n",
        "header = W.HTML('<h3>🧩 A1111 Extension Manager</h3>')\n",
        "cols   = W.HBox([W.Label('Extensions', layout=W.Layout(width='300px')),\n",
        "                   W.Label('Status',     layout=W.Layout(width='150px')),\n",
        "                   W.Label('Homepage')])\n",
        "\n",
        "# Build Recommended rows with checkboxes\n",
        "rows = []\n",
        "cbs, labels = [], []\n",
        "for item in RECOMMENDED:\n",
        "    cb = W.Checkbox(value=bool(item.get('enable', False)))\n",
        "    cbs.append(cb)\n",
        "    status = W.Label('✅ Installed' if installed_in_code(item['url']) else '❌ Not Installed',\n",
        "                     layout=W.Layout(width='150px'))\n",
        "    labels.append(status)\n",
        "    link = W.HTML(f"<a href='{item['url']}' target='_blank'>GitHub</a>")\n",
        "    row  = W.HBox([cb, W.Label(item['name'], layout=W.Layout(width='300px')), status, link])\n",
        "    rows.append(row)\n",
        "\n",
        "extra_label = W.HTML('<b>Extra Git repo URLs (one per line):</b>')\n",
        "extra_text  = W.Textarea(placeholder='https://github.com/user/repo\nhttps://github.com/other/repo', layout=W.Layout(height='90px'))\n",
        "btn_install = W.Button(description='Install Selected', button_style='success', icon='check')\n",
        "btn_refresh = W.Button(description='Refresh Status',  button_style='',       icon='refresh')\n",
        "btn_restart = W.Button(description='Restart A1111',   button_style='warning', icon='repeat')\n",
        "out = W.Output()\n",
        "\n",
        "def refresh_status():\n",
        "    # Defaults section status is rendered only in output; recommended labels updated live\n",
        "    for item, lab in zip(RECOMMENDED, labels):\n",
        "        lab.value = '✅ Installed' if installed_in_code(item['url']) else '❌ Not Installed'\n",
        "\n",
        "def on_install_clicked(_):\n",
        "    with out:\n",
        "        clear_output()\n",
        "        print('Installing…')\n",
        "        # recommended\n",
        "        for cb, item in zip(cbs, RECOMMENDED):\n",
        "            if cb.value:\n",
        "                install_or_update(item['url'], CODE_EXT_DIR)\n",
        "        # extra urls\n",
        "        extra = [u.strip() for u in extra_text.value.splitlines() if u.strip()]\n",
        "        for url in extra:\n",
        "            install_or_update(url, CODE_EXT_DIR)\n",
        "        refresh_status()\n",
        "        print('\nDone. In WebUI: Settings → Reload UI (or use Restart button).')\n",
        "\n",
        "def on_refresh_clicked(_):\n",
        "    refresh_status()\n",
        "\n",
        "def on_restart_clicked(_):\n",
        "    with out:\n",
        "        clear_output()\n",
        "        print('Restarting A1111 via supervisord…')\n",
        "        run(['supervisorctl', '-c', '/etc/supervisord.conf', 'restart', 'a1111'])\n",
        "        print('Sent restart. If the UI doesn\'t reconnect automatically, refresh your browser.')\n",
        "\n",
        "btn_install.on_click(on_install_clicked)\n",
        "btn_refresh.on_click(on_refresh_clicked)\n",
        "btn_restart.on_click(on_restart_clicked)\n",
        "\n",
        "# ---- Auto-install DEFAULTS once per run ----\n",
        "_installed_any_default = False\n",
        "for d in DEFAULT_EXTS:\n",
        "    if not installed_in_code(d['url']):\n",
        "        install_or_update(d['url'], CODE_EXT_DIR)\n",
        "        _installed_any_default = True\n",
        "\n",
        "# Pretty log for defaults\n",
        "defaults_log_lines = []\n",
        "for d in DEFAULT_EXTS:\n",
        "    defaults_log_lines.append(f"- {d['name']}: {'✅ Installed' if installed_in_code(d['url']) else '❌ Not Installed'}")\n",
        "defaults_summary = '<br>'.join(defaults_log_lines)\n",
        "defaults_html = W.HTML(f"""
        <div style='margin:8px 0 12px 0;'>
          <b>Defaults (auto-installed):</b><br>
          {defaults_summary}
        </div>
        """)\n",
        "\n",
        "refresh_status()\n",
        "ui = W.VBox([
",
        "    header,
",
        "    defaults_html,
",
        "    cols,
",
        "    *rows,
",
        "    W.HTML('<hr>'),
",
        "    extra_label,
",
        "    extra_text,
",
        "    W.HBox([btn_install, btn_refresh, btn_restart], layout=W.Layout(gap='10px')),
",
        "    out
",
        "])\n",
        "display(ui)\n

### Notes
- **Defaults** are installed automatically every time you run the cell (it’s safe; existing repos are updated).
- **Recommended** are controlled by checkboxes; click **Install Selected** to install/update them.
- Paste any extra GitHub repo URLs and install them in the same pass.
- If extensions don’t show immediately, click **Restart A1111** here or **Settings → Reload UI** in WebUI.